# Intro To Tensorflow 

Notes based off of Data Incubator classes

In [5]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()

In [2]:
import tensorflow as tf
import numpy as np

## Introduction

In Tensorflow, coputations are conceptuatlized as directed and acyclic graphs. The nodes are operations.

Example:  
$x = 1$  
$y = 5$  
$sum\_ = x + y$  

In [7]:
x = tf.constant(1, name="input_x")
y = tf.constant(5, name="input_y")
sum_ = tf.add(x, y, name="sum")

In [9]:
sess = tf.Session()
sess.run(sum_)

6

Equivalently:

In [10]:
sum2 = x + y

In [11]:
sess.run(sum2)

6

Note: to check equality of two tensors, must use "tf.equal(x,y)" and not "=="

#### Repeating calculations

You can feed new values

In [13]:
sess.run(sum_, feed_dict={x:100, y: 500})

600

Instead of explicitly defining constants as we did earlier, we can use placeholders:

In [16]:
xp = tf.placeholder(dtype=tf.int32)
sum_p = tf.add(xp, y)
sess.run(sum_p, {xp: 100})

105

#### Matrices

Think of tensors as an n-dimensional array of numbers (though perhaps not mathematically totally accurate, this is a fine definition for purposes of TensorFlow)

In [18]:
tf.constant([[1,2,3],[4,5,6]]).shape

TensorShape([Dimension(2), Dimension(3)])

In [25]:
mat = tf.placeholder(dtype=tf.int32, shape=(2,3))
sess.run(mat, {mat:[[1,2,3],[4,5,6]]})

array([[1, 2, 3],
       [4, 5, 6]], dtype=int32)

The placeholder matrix below uses a dimension of None to indicate that the tensor can have arbitrary size along that dimension.

In [27]:
mat2 = tf.placeholder(dtype=np.int32, shape=(2, None))
print(sess.run(mat2, {mat2: [[1],[2]]}))
print(sess.run(mat2, {mat2: [[1,2,3],[4,5,6]]}))

[[1]
 [2]]
[[1 2 3]
 [4 5 6]]


Matrix multiplication: (Default is element-wise, use tf.matmul)

In [28]:
sess.run(mat2 * mat2, {mat2: [[1,2],[3,4]]})

array([[ 1,  4],
       [ 9, 16]], dtype=int32)

In [29]:
sess.run(tf.matmul(mat2, mat2), {mat2: [[1,2],[3,4]]})

array([[ 7, 10],
       [15, 22]], dtype=int32)

Summing rows or columns

In [30]:
sess.run(tf.reduce_sum(mat, axis=0), {mat: [[1,2,3],[4,5,6]]})

array([5, 7, 9], dtype=int32)

In [31]:
sess.run(tf.reduce_sum(mat, axis=1), {mat: [[1,2,3],[4,5,6]]})

array([ 6, 15], dtype=int32)

In [32]:
sess.run(tf.reduce_sum(mat), {mat: [[1,2,3],[4,5,6]]})

21

## Graphs

Best practice is to create a new graph for each set of operations

In [34]:
g = tf.get_default_graph()
g

In [35]:
g.get_operations()

[<tf.Operation 'input_x' type=Const>,
 <tf.Operation 'input_y' type=Const>,
 <tf.Operation 'sum' type=Add>,
 <tf.Operation 'add' type=Add>,
 <tf.Operation 'Placeholder' type=Placeholder>,
 <tf.Operation 'Add' type=Add>,
 <tf.Operation 'Placeholder_1' type=Placeholder>,
 <tf.Operation 'Add_1' type=Add>,
 <tf.Operation 'Const' type=Const>,
 <tf.Operation 'Placeholder_2' type=Placeholder>,
 <tf.Operation 'Placeholder_3' type=Placeholder>,
 <tf.Operation 'Placeholder_4' type=Placeholder>,
 <tf.Operation 'Placeholder_5' type=Placeholder>,
 <tf.Operation 'Placeholder_6' type=Placeholder>,
 <tf.Operation 'mul' type=Mul>,
 <tf.Operation 'MatMul' type=MatMul>,
 <tf.Operation 'Sum/reduction_indices' type=Const>,
 <tf.Operation 'Sum' type=Sum>,
 <tf.Operation 'Sum_1/reduction_indices' type=Const>,
 <tf.Operation 'Sum_1' type=Sum>,
 <tf.Operation 'Const_1' type=Const>,
 <tf.Operation 'Sum_2' type=Sum>]

In [38]:
g2 = tf.Graph()

In [39]:
with g2.as_default():
    a = tf.constant(12, name="constant_a")
    b = tf.constant(16, name="constant_b")
    sum_g2 = a + b

In [40]:
g2.get_operations()

[<tf.Operation 'constant_a' type=Const>,
 <tf.Operation 'constant_b' type=Const>,
 <tf.Operation 'add' type=Add>]

## Sessions

Each session is associated with a particular graph.
"A session abstracts the environment in which a graph is executed"

In [43]:
sess2 = tf.Session(graph=g2)
sess2.run(sum_g2)

28

In [45]:
sess2.close()

Helper function, it closes current session and opens a new one, resetting the default graph.

In [46]:
def reset_tf():
    global sess
    if sess:
        sess.close()
    tf.reset_default_graph()
    sess = tf.Session()